In [ ]:
#Code No.1
%matplotlib nbagg
%matplotlib inline
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2 ** 128
import numpy as np
import cv2
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import time
import math
import matplotlib
from glob import glob
import openpyxl
from openpyxl import Workbook
import pprint
import os
import pandas as pd
from itertools import zip_longest
import itertools
from scipy.stats import norm
import matplotlib.animation as animation
from IPython import display
from sklearn import mixture
from matplotlib.animation import PillowWriter
from PIL import Image
import sys
# detect contours in an image
def contours_area_center_ellipse_fitting(num,img,mini_area,max_area):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    num_x_y_ellipse_area_deg, area_arr  = [],[]
    for i, cnt in enumerate(contours):
        area = cv2.contourArea(cnt)
        if max_area >= area >= mini_area:#Smaller masses are not considered bacteria.
            mu = cv2.moments(cnt)#Center of Gravity
            if mu["m00"] != 0:
                x,y= float(mu["m10"]/mu["m00"]) , float(mu["m01"]/mu["m00"])
                area_arr.append([area])
                ellipse_arr = cv2.fitEllipse(cnt)
            num_x_y_ellipse_area_deg.append([num]+[x, y]+list(ellipse_arr[1:3]))
        else:
            pass
    return area_arr, num_x_y_ellipse_area_deg #Return [frame No, Centroid coordinate,Major and Minor axes of the fitted ellipsoid, angle]
# Verify correspondence between objects in past and present frames
def judgement_object(old_list,new_list,dis_th,x_um_pix,y_um_pix):
    temp_list, next_input, output = [],[],[]
    compared_list, changed_list = old_list.copy(), old_list.copy()
    for i, n_list in enumerate(new_list):
        for j, o_list in enumerate(old_list):
            torf = False
            loss = np.sqrt(((o_list[-4]-n_list[1])*x_um_pix)**2 + ((o_list[-3]-n_list[2])*y_um_pix)**2)
            if loss < dis_th:#If the distance traveled between frames is shorter than the threshold, the cells are considered to be identical.
                torf = True
                o_list += n_list
                changed_list[j] = o_list
                next_input.append(o_list)
                break
            else:
                pass
        if torf == False:
            temp_list.append(n_list)
    for k, list_k in enumerate(compared_list):
        if changed_list[k] == list_k:
            output.append(list_k)
    next_input += temp_list
    return next_input, output
def remove_objects(img,lower_size=None,upper_size=None):#Remove objects of size outside the specified range
    # find all objects
    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(img)
    sizes = stats[1:, -1]
    _img = np.zeros((labels.shape))
    # process all objects, label=0 is background, objects are started from 1
    for i in range(1, nlabels):
        # remove small objects
        if (lower_size is not None) and (upper_size is not None):
            if lower_size < sizes[i - 1] and sizes[i - 1] < upper_size:
                _img[labels == i] = 255
        elif (lower_size is not None) and (upper_size is None):
            if lower_size < sizes[i - 1]:
                _img[labels == i] = 255
        elif (lower_size is None) and (upper_size is not None):
            if sizes[i - 1] < upper_size:
                _img[labels == i] = 255
    return _img
def points_plot(img,list_x_y):#Draw a plot to show tracking
    for i in list_x_y:
        iter_i = iter(i)
        c = (0,255,0)
        for j,k,l,m,n in zip(iter_i,iter_i,iter_i,iter_i,iter_i):
            img_ = cv2.circle(img,(int(k),int(l)),1,c,2)
    return img_
def reg1dim(x, y):
    a = np.dot(x, y)/ (x ** 2).sum()
    return a

In [ ]:
#Code No.2
path = "C:"#Specify the directory containing the video to be analyzed
os.chdir(path)crawling_movies = glob("./*.avi")
print(crawling_movies)

#Specify the video No.to be analyzed
movie_number =1
print(crawling_movies[movie_number])

#pixel-length calibration parameters(um/pixel)
x_um_pix,y_um_pix,mag = 0.170,0.187,1

#Save as video file
write_video=False#True
movie_format='avi'#'mp4'

#Parameter setting for image analysis
pair_name = 'Data Name' #Specify the data name; the same name must be included in the Excel file name (see below)
history, dist2Threshold= 200, 600 #history=T, dist2Threshold=Mahalanobis distance threshold 200, 10
kernel = np.ones((5,5),np.uint8) #dilation and erosion parameters
dis_th=3 #Threshold for tracking　
mini_area1, max_area1 = int(6*mag), int(10000*mag) #Removal of noise and too large particles before dilation and erosion
mini_area2, max_area2 = int(6*mag), int(10000*mag) #Removal of noise and too large particles after dilation and erosion (pixel^2)
idx = crawling_movies[movie_number].find(".avi")
movie_name = crawling_movies[movie_number][2:idx]
#The following Microsoft Excel files must be prepared in the same directory as the movie files to be analyzed:
#crawling_Data Name.xlsx (Data name is the same as the one specified as pair name (above); If swimming data will be analyzed,"swimming_Data Name")
#data_cum_dis.xlsx (cumulative displacement vs time)
#data_dis.xlsx (migration distance/frame)
#data_msd.xlsx (mean square displacement vs time used for diffusion analysis)
#data_speed.xlsx (summary of speed data)
#data_x,y.xlsx (2D tracking data of individual bacteria)

In [ ]:
#Code No.3
#Read a movie file
cap = cv2.VideoCapture(crawling_movies[movie_number])
w, h = cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)
movie_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
window_size = (int(w), int(h))
if write_video == True:
    frame_rate = int(fps) # frame rate
    window_size = (int(w), int(h)) # movie aspect ratio
    if movie_format=='avi':
        fmt = cv2.VideoWriter_fourcc('I', 'Y', 'U', 'V')
        writer = cv2.VideoWriter(path + 'name.avi',\
                             apiPreference=0, fourcc=fmt, fps=int(frame_rate), frameSize=window_size, isColor=True)
        writer1 = cv2.VideoWriter(path + 'trace.mp4',apiPreference=0, fourcc=fmt2, fps=frame_rate, frameSize=window_size, isColor=False)
    if movie_format=='mp4':
        fmt = cv2.VideoWriter_fourcc(*'mp4v')
        writer = cv2.VideoWriter(path + 'name.mp4',\
                                 apiPreference=0, fourcc=fmt, fps=int(frame_rate), frameSize=window_size, isColor=True)
fgbg = cv2.createBackgroundSubtractorKNN(history=history,dist2Threshold = dist2Threshold,detectShadows=False)
output_list = []
i = 0
pbar = tqdm(total=int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
while(1):
    ret, frame = cap.read()
    if not ret:
        break
    gray_ = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.blur(gray_, (3,3))
    mask = fgbg.apply(gray)
    if i == 0:
        num = 0
        pass
    elif i == 1:
        area, old_list = contours_area_center_ellipse_fitting(i,mask,mini_area=mini_area1,max_area=max_area1)
    else:
        mask1 = remove_objects(img=mask,lower_size=mini_area1,upper_size=max_area1).astype(np.uint8)
        mask2 = cv2.morphologyEx(mask1,cv2.MORPH_CLOSE,kernel, iterations=1).astype(np.uint8)
        mask3 = remove_objects(img=mask2,lower_size=mini_area2,upper_size=max_area2)
        contours, hierarchy = cv2.findContours((mask3).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        img2 = mask3.copy()
        for j in range(len(contours)):
            if len(contours[j])>5:  #5
                axis,length,degree=cv2.fitEllipse(contours[j])
                if 0.99<(length[0]/length[1])<1.01:   #0.99 < <1.01
                    img2 = cv2.drawContours(img2, [contours[j]], 0, 0, -1)  #アスペクト比が1
                else:
                    img2 = cv2.drawContours(img2, [contours[j]], 0, 255, -1)
            else: img2 = cv2.drawContours(img2, [contours[j]], 0, 0, -1)
        mask_th = img2.astype(np.uint8).copy()
        m = np.where(mask2==255,255,gray)
        if old_list == []:
            area, old_list = contours_area_center_ellipse_fitting(i,mask_th,mini_area=mini_area2,max_area=max_area2)
        else:
            area, new_list = contours_area_center_ellipse_fitting(i,mask_th,mini_area=mini_area2,max_area=max_area2)
            if new_list == []:
                output_list += old_list
                old_list == []
            else:
                old_list, output = judgement_object(old_list=old_list, new_list=new_list,\
                                                    dis_th=dis_th,x_um_pix = x_um_pix,y_um_pix = y_um_pix)
                output_list += output
                pil_image = Image.fromarray(mask_th.copy())
                img_point = np.asarray(pil_image.convert("L").convert("RGB"), np.uint8)
                pil_image_m = Image.fromarray(m.copy())
                img_point_m = np.asarray(pil_image_m.convert("L").convert("RGB"), np.uint8)
                mask_output = np.where(mask_th==255,255,gray)
                pil_image_o = Image.fromarray(mask_output)
                img_point_o = np.asarray(pil_image_o.convert("L").convert("RGB"), np.uint8)
                img_point_plot = points_plot(img_point_m.copy(), old_list)
                gray_o = np.asarray(Image.fromarray(gray).convert("L").convert("RGB"), np.uint8)
                mask1 = np.asarray(Image.fromarray(mask1).convert("L").convert("RGB"), np.uint8)
                mask2 = np.asarray(Image.fromarray(mask2).convert("L").convert("RGB"), np.uint8)
                mask3 = np.asarray(Image.fromarray(mask3).convert("L").convert("RGB"), np.uint8)
                images_1, images_2 = cv2.hconcat([gray_o,mask1,mask2]),cv2.hconcat([gray_o,img_point_plot])
                cv2.imshow('window',cv2.vconcat([images_2]))
                if write_video == True:
                    writer.write(gray)
                    writer.write(img_point_plot1)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    i += 1
    pbar.update(1)
if write_video == True:# close file
    writer1.release()
    #writer2.release()
cap.release()
cv2.destroyAllWindows()

In [ ]:
#Code No.4
def displacement(cell,x_um_pix,y_um_pix):
    x=np.array(cell[1::5]).flatten()
    x=list(map(lambda a: a*x_um_pix, x))
    y=np.array(cell[2::5]).flatten()
    y=list(map(lambda a: a*y_um_pix, y))
    t = []
    displacement = []
    num=0
    for j,k in zip(x,y):
        if num == 0:
            x_base,y_base=j,k
        x_new, y_new = j-x_base, k-y_base
        r_square = x_new**2 + y_new**2
        r = math.sqrt(r_square)
        t.append(num / fps)
        displacement.append(r)
        num += 1
    return t,x,y,displacement
#parameter setting
min_f_len = 5 #frames
min_dis=3 #micrometer
max_f_len = max([len(x[::5]) for x in output_list])
cells = [l for l in output_list if max_f_len >= len(l[::5]) > min_f_len and max(displacement(cell=l,x_um_pix=x_um_pix,y_um_pix=y_um_pix)[3]) > min_dis]
cells = list(map(list, set(map(tuple, cells))))#pixel
print("The number of measured bactera:{} bacteria".format(len(cells)))
print("The longet tracking time: {}frames".format(max([len(x[::5]) for x in cells])))

In [ ]:
#Code No.5
cells = sorted(cells)
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111)
colors = list(matplotlib.colors.XKCD_COLORS.items())[:len(cells)]
start_number=0
cells_separate = cells[start_number:start_number+len(cells)]#
for i,(cell,color) in enumerate(zip(cells_separate,colors)):
    t,x,y,dis = displacement(cell=cell,x_um_pix=x_um_pix,y_um_pix=y_um_pix)
    # data plot
    ax.plot(t, dis, linewidth=4,color=color[1],label=f"{i}")
ax.set_xlabel("time (s)",fontsize=15),ax.set_ylabel("displacement (um)",fontsize=15)
ax.tick_params(axis='x', labelsize=15),ax.tick_params(axis='y', labelsize=15)
plt.show()
plt.close()

In [ ]:
#Code No.6
fig = plt.figure(figsize=(15,20))
ax2 = fig.add_subplot(211)
ax3 = fig.add_subplot(212)
plt.xticks(fontsize=30),plt.yticks(fontsize=30)
font_dict = dict(weight="bold",size=30)
colors = list(matplotlib.colors.XKCD_COLORS.items())[:len(cells)]
for i,(cell,color) in enumerate(zip(cells,colors)):
    t,x,y,dis = displacement(cell=cell,x_um_pix=x_um_pix,y_um_pix=y_um_pix)
    ax2.text(x[0],y[0],f"{i+start_number}",dict(weight="bold",size=30,color=color[1]))
    ax2.scatter(x, y,linewidth=4,color=color[1],label=f"{i}",s=5)
    ax3.text(x[0],y[0],f"{i+start_number}",dict(weight="bold",size=30,color=color[1]))
    ax3.scatter(x, y,linewidth=4,color=color[1],label=f"{i}",s=5)
cap = cv2.VideoCapture(crawling_movies[movie_number])
ret, frame = cap.read()
img = np.array(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
img = cv2.resize(img, (int(w*x_um_pix), int(h*y_um_pix)))
ax2.imshow(img)
# graph display
#ax.set_xlabel("time (s)",fontsize=30),ax.set_ylabel("displacement (um)",fontsize=30)
ax2.set_xlabel("x (um)",fontsize=30),ax2.set_ylabel("y (um)",fontsize=30)
ax3.set_xlabel("x (um)",fontsize=30),ax3.set_ylabel("y (um)",fontsize=30)
ax2.tick_params(axis='x', labelsize=30),ax2.tick_params(axis='y', labelsize=30)
ax3.tick_params(axis='x', labelsize=30),ax3.tick_params(axis='y', labelsize=30)
ax3.set_xlim(0, w*x_um_pix),ax3.set_ylim(h*y_um_pix, 0)
ax3.legend(loc='upper left', bbox_to_anchor=(0, -0.1,), ncol=10,fontsize=30,markerscale=5)
plt.show()
plt.close()

In [ ]:
#Code No.7
cells_pickup=[]
# Extract the available data; write the tracking No to be anlyzed in the following stepes
#[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,\
#31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,\
#71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103]

crawling_cell_no = [0,1,2]
if len(crawling_cell_no) != 0:
    for i,num in enumerate(crawling_cell_no):
        cells_pickup.append(cells[num])

In [ ]:
#Code No.8
#Save data
genre = 'crawling' #'swimming' for analyzing swimming data
wb = openpyxl.load_workbook(path+"{0}_{1}.xlsx".format(genre,pair_name))
ws = wb.create_sheet(title=movie_name)
ws["A1"] = "info"
ws["B1"] = pair_name
ws["A4"] = "time(s)"
max_row = 0
ws.cell(row=2,column=1).value = genre
count=0
for cell in cells_pickup:
    ws.cell(row=3,column=2*count+2).value = f"no.{count+1}"
    ws.cell(row=4,column=2*count+2).value = "x(um)"
    ws.cell(row=4,column=2*count+3).value = "y(um)"
    ws.cell(row=3,column=len(cells_pickup)*2+count+2).value = f"no.{count+1}"
    ws.cell(row=4,column=len(cells_pickup)*2+count+2).value = "angle(degree)"
    ws.cell(row=3,column=len(cells_pickup)*3+count+2).value = f"no.{count+1}"
    ws.cell(row=4,column=len(cells_pickup)*3+count+2).value = "frame_num"
    frame_number_range=np.array(cell[0::5])
    x=np.array(cell[1::5]).flatten()
    x_um=x*x_um_pix
    y=np.array(cell[2::5]).flatten()
    y_um=y*y_um_pix
    angle=cell[4::5]
    count_2 = 0
    for x_point,y_point,angle_point,f_num\
        in zip_longest(x_um,y_um,angle,frame_number_range):
        ws.cell(count_2+5,2*count+2,x_point)
        ws.cell(count_2+5,2*count+3,y_point)
        ws.cell(count_2+5,len(cells_pickup)*2+count+2,angle_point)
        ws.cell(count_2+5,len(cells_pickup)*3+count+2,f_num)
        count_2 += 1
    t = []
    t = np.linspace(0,len(x)/fps,len(x))
    if max_row < len(x):
        max_row = len(x)
    count+=1
for i in range(0,max_row):
    spend_time = i/fps
    ws.cell(row=i+5,column=1,value=spend_time)
#save with the specified name
wb.save(path+"{0}_{1}.xlsx".format(genre,pair_name))

In [ ]:
#Code No.9
#data classification
#genre = 'crawling' 
mode_writer = "a"
idx_ = pair_name.find("_")
cell_name = pair_name[:idx_]
species_name = pair_name[idx_+1:]
if os.path.isfile(path+"{0}_{1}.xlsx".format(genre,pair_name)):
    wb = openpyxl.load_workbook(path + "{0}_{1}.xlsx".format(genre,pair_name))
    ws_list = wb.sheetnames
    #get the longest data lenght (i.e., tracking frames) in the sheet
    max_data_length_list=[]
    for h in ws_list:
        if h=="Sheet1":
            print('Error!Remove Sheet1')
            sys.exit(1)
        else:
            ws = wb[h]
            max_data_length_list.append(ws.max_row)
    #Get the time axis from the longest data
    time=[]
    max_length = max(max_data_length_list)
    sheet_number = max_data_length_list.index(max_length)
    ws_time = wb.worksheets[sheet_number]
    for cols in ws_time.iter_cols(min_row=5, min_col=1, max_col=1):
        for cell in cols:
            time.append(cell.value)
    df_time = pd.DataFrame(time,columns=["time(s)"])
    count=0
    #Get the data from the sheet
    for i,name in enumerate(ws_list):
        ws = wb[name]
        data_number = (ws.max_column - 1) // 4
        for j in np.arange(0,data_number):
            data=[]
            angle_data=[]
            start_frame_data=[]
            for cols in ws.iter_rows(min_row=5, min_col=j*2+2, max_col=j*2+3):
                for cell in cols:
                    data.append(cell.value)
            for cols in ws.iter_rows(min_row=5, min_col=data_number*2+j+2, max_col=data_number*2+j+2):
                for cell in cols:
                    angle_data.append(cell.value)
            for cols in ws.iter_rows(min_row=5, min_col=data_number*3+j+2, max_col=data_number*3+j+2):
                for cell in cols:
                    start_frame_data.append(cell.value)
            x = pd.Series(data[::2],name=f'x{i}_{j}(um)').dropna().reset_index(drop=True)
            y = pd.Series(data[1::2],name=f'y{i}_{j}(um)').dropna().reset_index(drop=True)
            angle = pd.Series(angle_data,name=f'angle{i}_{j}(deg)').dropna().reset_index(drop=True)
            start_frame = pd.Series(start_frame_data, name = f'frame{i}_{j}')
            if count == 0:
                df_x_y = pd.concat([df_time,x,y],axis=1)
                df_angle = pd.concat([df_time,angle],axis=1)
                df_start_frame = pd.concat([df_time,start_frame],axis=1)
            else:
                df_x_y = pd.concat([df_x_y,x,y],axis=1)
                df_angle = pd.concat([df_angle,angle],axis=1)
                df_start_frame = pd.concat([df_start_frame,start_frame],axis=1)
            count += 1
    with pd.ExcelWriter(\
         path + 'data_x,y.xlsx'\
         , engine="openpyxl", mode=f'{mode_writer}', if_sheet_exists='replace') as writer:
         df_x_y.to_excel(writer, sheet_name=pair_name,index=False, header=True) #If angle data is needed, df_angle
else:
    pass
print('end')

In [ ]:
#Code No.10
range_num = 5
if range_num % 2 == 0:
    start,end = range_num/2,range_num/2 - 1
else:
    start,end = range_num//2,range_num//2
wb = openpyxl.load_workbook(path + 'data_x,y.xlsx')
ws_list = wb.sheetnames
data_x_y = pd.read_excel(path+'data_x,y.xlsx',sheet_name=None,index_col=None,header=0)
pair_names = list(data_x_y.keys())
for pair_name in pair_names:
    if pair_name=="Sheet1":
        print('Error!Remove Sheet1')
        sys.exit(1)
    print(pair_name)
    columns_length = (len(data_x_y[pair_name].columns)-1)//2
    df_time = data_x_y[pair_name].iloc[:,0]
    df_delta_time = df_time[:-range_num]
    df_delta_time.name = 'Δtime(s)'
    df_log_time = np.log10(df_time[1:-range_num]).reset_index(drop=True)
    df_log_time.name = 'Δtime(s)_log10'
    count=0
    for number in range(columns_length):
        msd_list,judge,a_list = [],[],[]
        df_x_y_each = data_x_y[pair_name].iloc[:,2*number+1:2*number+3].dropna()
        df_time_each = data_x_y[pair_name].iloc[:len(df_x_y_each),0]
        row_length = len(df_time_each)
        x_y_offset = df_x_y_each - df_x_y_each.iloc[0,:]
        x_y_ave = df_x_y_each.rolling(range_num, center=True).mean().iloc[start:-end].reset_index(drop=True)
        x_y_ave_offset = x_y_ave - x_y_ave.iloc[0,:]
        delta_x_y = x_y_ave.diff().dropna().reset_index(drop=True)
        delta_r = np.sqrt(delta_x_y.iloc[:,0]**2 + delta_x_y.iloc[:,1]**2)
        cum_sum_r = pd.Series([0] + list(itertools.accumulate(delta_r)))
        displacement = np.sqrt(x_y_offset.iloc[:,0]**2 + x_y_offset.iloc[:,1]**2)
        speed=reg1dim(df_time.iloc[:len(cum_sum_r)],cum_sum_r.iloc[:len(cum_sum_r)])
        displacement.name,cum_sum_r.name,delta_x_y.name=f"{count}",f"{count}",f"{count}"
        x_max,y_max = max(x_y_offset.iloc[:,0].max(),abs(x_y_offset.iloc[:,0].min())),\
                      max(x_y_offset.iloc[:,1].max(),abs(x_y_offset.iloc[:,1].min()))
        max_value,kizami = math.ceil(max(x_max,y_max)),1
        bins=list(np.arange(-max_value,max_value+kizami,kizami))
        (X, Y) = np.meshgrid(bins,bins)
        H = np.histogram2d(x_y_offset.iloc[:,0],x_y_offset.iloc[:,1],bins)
        H_den = np.histogram2d(x_y_offset.iloc[:,0],x_y_offset.iloc[:,1],bins,density=True)
        X_hist_,Y_hist_ = np.histogram(x_y_offset.iloc[:,0],bins),np.histogram(x_y_offset.iloc[:,1],bins)
        range_ = [list(X[0][::-1])]*(max_value*2+1)
        hist1d_data = pd.concat([pd.DataFrame(bins[:-1]).rename(columns={0:'bin'}),pd.DataFrame(X_hist_[0]).rename(columns={0:'x'}),\
                   pd.DataFrame(Y_hist_[0]).rename(columns={0:'y'})],axis=1)
        for i in range(len(bins)-1):
            a=pd.DataFrame(list(Y[::-1][i][:-1])).reset_index(drop=True).rename(columns={0:'x'})
            b=pd.DataFrame(range_[0][:-1]).reset_index(drop=True).rename(columns={0:'y'})
            c = pd.DataFrame(H[0]).iloc[:,i:i+1].rename(columns={i:'H'})
            d = pd.DataFrame(H_den[0]).iloc[:,i:i+1].rename(columns={i:'H_den'})
            hist2d = pd.concat([a,b,c,d],axis=1)
            if i == 0:
                hist2d_data = hist2d
            else:
                hist2d_data = pd.concat([hist2d_data,hist2d],axis=0)
        hist2d_data.rename(columns={'x': f'x_{count}','y': f'y_{count}','H': f'H_{count}','H_den': f'H_den_{count}'}, inplace=True)
        for l in range(row_length-range_num):
            msd = (x_y_offset.iloc[:,0].diff(l).dropna() ** 2).mean() + (x_y_offset.iloc[:,1].diff(l).dropna() ** 2).mean()
            msd_list.append(msd)
        msd_data = pd.Series(msd_list,name=f"{count}")
        msd_log_data = np.log10(msd_data.iloc[1:])
        msd_log_data.name=f"{count}"
        if count == 0:
            df_speed = pd.Series(speed)
            df_msd = pd.concat([df_delta_time,msd_data],axis=1)
            df_msd_log = pd.concat([df_log_time,msd_log_data],axis=1)
            df_dis = pd.concat([df_time,displacement],axis=1)
            df_cum_dis = pd.concat([df_time[:-range_num+1],cum_sum_r],axis=1)
            df_delta = pd.concat([df_time[start+1:-end].reset_index(drop=True),delta_x_y],axis=1)
            df_hist2d = hist2d_data
            df_hist1d = hist1d_data
        else:
            df_speed = pd.concat([df_speed,pd.Series(speed)],axis=0)
            df_msd = pd.concat([df_msd,msd_data],axis=1)
            df_msd_log = pd.concat([df_msd_log,msd_log_data],axis=1)
            df_dis = pd.concat([df_dis,displacement],axis=1)
            df_cum_dis = pd.concat([df_cum_dis,cum_sum_r],axis=1)
            df_delta = pd.concat([df_delta,delta_x_y],axis=1)
            df_hist2d = pd.concat([df_hist2d.reset_index(drop=True),hist2d_data.reset_index(drop=True)],axis=1)
            df_hist1d = pd.concat([df_hist1d,hist1d_data],axis=1)
        count += 1
    with pd.ExcelWriter(path+'data_msd.xlsx', engine="openpyxl", mode=f'{mode_writer}', if_sheet_exists='replace') as writer:
         df_msd.to_excel(writer, sheet_name=pair_name,index=False, header=True)
    with pd.ExcelWriter(path+'data_dis.xlsx', engine="openpyxl", mode=f'{mode_writer}', if_sheet_exists='replace') as writer:
         df_dis.to_excel(writer, sheet_name=pair_name,index=False, header=True)
    with pd.ExcelWriter(path+'data_cum_dis.xlsx', engine="openpyxl", mode=f'{mode_writer}', if_sheet_exists='replace') as writer:
         df_cum_dis.to_excel(writer, sheet_name=pair_name,index=False, header=True)
    with pd.ExcelWriter(path+'data_speed.xlsx', engine="openpyxl", mode=f'{mode_writer}', if_sheet_exists='replace') as writer:
         df_speed.to_excel(writer, sheet_name=pair_name,index=False, header=True)
else: pass